In [1]:
from utils import get_nse_expiry_date

In [4]:
symbol = 'RELIANCE'
get_nse_expiry_date(symbol=symbol)

datetime.date(2024, 5, 30)

# Generate nearest date for nse expiry

## Consider holiday list

In [ ]:
import json
import pandas as pd

In [ ]:
from utils import get_nse_payload
def get_nse_holidays() -> set:

    """
    Gets holidays from nse website
    """

    url = "https://www.nseindia.com/api/holiday-master?type=trading"

    r = get_nse_payload(url)

    holiday_dict = json.loads(r.text).get('CBM')
    df_nse_holidays = pd.DataFrame.from_records(holiday_dict)

    out = set(pd.to_datetime(df_nse_holidays.tradingDate).dt.date)

    return out


In [ ]:
listOfNseHolidays = get_nse_holidays()

## Get weeklies expiry date

In [ ]:
import datetime
from datetime import date, timedelta
import calendar


In [ ]:
def get_weekday_int(weekday_name: str) -> int:
    """Gives the integer of the weekday string given"""

    weekday_name = weekday_name.upper()

    weekdays = {name.upper(): i for i, name in enumerate(calendar.day_name)}


    # takes care of abbreviated weekday_name
    weekday_name = [k for k, v in weekdays.items() if weekday_name.upper() in k][0]

    weekday_int = weekdays.get(weekday_name)

    return weekday_int

In [ ]:
def get_weekday_date(weekday_name: str):

    """
    Gets date of the weekday provided in `str`
    """

    today = date.today()
    target_weekday = get_weekday_int(weekday_name)
    days_to_add = (target_weekday - today.weekday()) % 7

    return today + timedelta(days=days_to_add)


In [ ]:
def get_nse_weeklies_expiry_date() -> datetime.date:
    """Gets nse weeklies expiry for NSE index options"""

    weekly_expiry_date = get_weekday_date("Thu")

    if weekly_expiry_date in listOfNseHolidays:
        weekly_expiry_date = weekly_expiry_date - timedelta(days=1)

    return weekly_expiry_date

## Get Monthlies Expiry Date

In [ ]:
def get_nse_monthly_expiry_date() -> datetime.date:

    """Gets the nse monthly expiry date adjusted for holidays in this year"""
    
    today = today = datetime.date.today()

    # Get the last day of the month
    last_day_of_month = calendar.monthrange(today.year, today.month)[1]

    # Get month from the weekly expiry date
    month = get_nse_weeklies_expiry_date().month

    # Make last date of the month

    last_date_of_month = date(today.year, month, last_day_of_month )


    day_of_expiry = get_weekday_int("Thu") # Usual day of expiry
    weekday_delta = last_date_of_month.weekday() - day_of_expiry

    if weekday_delta >= 0:
        expiry_date = last_date_of_month - timedelta(weekday_delta)
    else:
        expiry_date = last_date_of_month - timedelta(7 - abs(weekday_delta))

    
    # Adjust for NSE holidays
    listOfNseHolidays = get_nse_holidays()

    if expiry_date in listOfNseHolidays:
        expiry_date = expiry_date - timedelta(days=1)

    return expiry_date



In [ ]:
get_nse_monthly_expiry_date()

## Gets nearest expiry date for an nse symbol

In [ ]:
import datetime
from utils import get_lots_from_nse
symlots = get_lots_from_nse()

In [ ]:
def this_is_nse_index(symbol: str) -> bool:
    return True if 'NIFTY' in symbol else False
symbol = 'NIFTY'

def get_nse_expiry_date(symbol: str) -> datetime.date:

    """Gets nse expiry date for the symbol, adjusted to this year's nse holidays"""

    out = get_nse_weeklies_expiry_date()\
        if this_is_nse_index(symbol)\
            else get_nse_monthly_expiry_date()
    
    return out